In [ ]:
"""
Google Code Jam 2008 Round 1A A


それぞれのベクトルについて、その中で要素を自由に入れ替えられるので、一方は動かす必要はない
smallケースは要素数が最大でも８なので、組み合わせが8!=40320なので全探索もできる
largeケースは間に合わない
内積を小さくしたいので、各要素の積の結果が小さくなると嬉しい
なので、「xの中で大きい要素」と「ｙの中で小さい要素」の積を足していくと解ける
"""

x = list(map(int, input().split()))
y = list(map(int, input().split()))

x.sort()
y.sort(reverse=True)

ans = 0
for i in range(len(x)):
    ans += x[i]*y[i]

print(ans)

In [ ]:


"""
Google Code Jam 2009 Round2 A


数学的に解法を得るのは難しい
上から順に、「i番目にいられる（対角線より下に１をもってこれる）、一番i行目に近いものを上に持ってくる」を上から順に繰り返すと操作回数が最小になる
（上の行から順に最小操作で埋めていくため、全体で見ても最小操作回数になる）
"""

N = int(input())

M = [input() for _ in range(N)]

idx_last_one = [-1] * N
for i in range(N):
    for j in range(N):
        if M[i][j] == "1":
            idx_last_one[i] = j

ans = 0
for i in range(N):
    # 上から順に決定していく
    p = -1
    for j in range(i,N):
        # 対角線より上に１が来ないもので、一番iに近いものを選択
        # この操作方法では、最後の１が残りの中で一番左にあるものが選ばれるわけではないが、
        # 条件を満たすものができるし、今回の方法の方が移動回数が少なくできる
        if idx_last_one[j] <= i:
            p = j
            break
    # pの位置にあるものをiまで移動させる
    for j in range(p,i,-1):
        tmp = idx_last_one[j]
        idx_last_one[j] = idx_last_one[j-1]
        idx_last_one[j-1] = tmp
        ans += 1

print(ans)


In [ ]:

"""
GCJ 2009 Round 1C C Bribe the Prisoners 

だいぶ理解に時間がかかった。
考え方の出発点としては、囚人全体から選択肢を考えていっても分岐が多すぎてわからないので、小さい範囲から見ていった方がいいよね、というところか。
で、A1~A3の解放最小コスト、A2~A4の解放最小コスト、～、のように狭い範囲で全体要素の最小コストのとり方を求めたら、
次はA1~A4の解放最小コスト、A2~A5の解放最小コスト、～、のように範囲を広げていくことで全体が求まる。ここで上で求めた要素のコストを利用する
事象的には解放ごとに見る領域が小さくなっていくが、だからこそ小さい範囲から決めていけるということなのかもしれない

いかにDPの形に落とし込めるか、が難しい。
問題があった時に、小さい問題に分割して、それらの組み合わせを利用して大きな問題を表現というか解けないかをしっかり考える
"""


P = int(input())
Q = int(input())
A = list(map(int, input().split()))
# 端の選択肢を解放する場合のDPを書きやすくするために追加
A = [0] + A + [P+1]
INF = float("inf")
# dp[i][j]：A[i]より右かつ、A[j]より左にいる人で、対象を解放するのにかかる最小コスト
# 求めたいのはdp[0][-1]
dp = [[INF for _ in range(Q+2)] for _ in range(Q+2)]
#間に解放対象の人がいないとり方はコスト０
for i in range(Q+1):
    dp[i][i+1] = 0

# dp[i][i+1]には解放対象（Aのなかに入っているやつ）がいないので、
# dp[0][0+2]からはじめて、dp[1][1+2],...,dp[Q-1][Q+1],その次dp[0][0+3], dp[1][1+3],...
# のように間にいる解放対象の少ない順にみていくことで最後に全対象を解放した場合のコストdp[0][-1]が求まる
# 具体的に言うと、A_1からA_3間の解放にかかる最小コスト、A_2からA_4間の解放にかかる最小コスト、...
# のように全体を部分に切り分けて狭い範囲から最小コストを求めていくことで全体が決まる
for w in range(2,Q+2):
    for i in range(Q+2-w):
        j = i+w
        # tは今までにi~k,k~jの間にいる対象を解放するのにかかったコストのうちの最小値
        # ここのために狭い領域から調べていく
        t = INF
        for k in range(i+1, j):
            t = min(t, dp[i][k] + dp[k][j])
        """
        i~jの間のkを選んだ時、それがどのｋであってもA[j] - A[i] - 2分のコストがかかる
        また、tはi~k,k~jの間にいる（i,j,kを含まない）解放対象を解放するために必要な最小コスト
        """
        dp[i][j] = t + A[j] - A[i] - 2

print(dp[0][-1])


In [ ]:
"""
millionaire 2008 apac

とても難しい。
1ラウンドずつ進めて最後のMラウンド時点の確率を求めるのは無理
小さい例から始めて、残り1ラウンドの時、2ラウンドの時、...と考えると理解できる。
https://lvs7k.github.io/posts/2018/11/pccb-easy-7/
ここの例がわかりやすかった。
currがdp[2]、prevがdp[1]の行だと考えて、dp[2][1]を求める場合、
dp[2][1]の状態から1の分だけ賭けて勝つとdp[1][2]の状態に移れるので、dp[1][2]の確率にPをかけた確率がdp[2][1]の一部
また、dp[2][1]の状態から1の分だけ賭けて負けると、dp[1][0]の状態に移れるので、dp[1][0]の確率に1-Pをかけた確率がdp[2][1]の一部
これを取りうるjについて全て探して、dp[2][1]が最大になるものがdp[2][1]の答えになる

・指定以上のお金を入手したらそれ以上参加しなくてもお金を持って帰ることができる
・１ラウンド分だけ考えたときに、
　- その時点で目標金額の半分未満しか持っていなかったら、勝っても目標に達しないので確率０
　- その時点で目標金額の半分以上目標未満をもっていれば、勝ったら持って帰ることができるのえｄ、確率P
　- その時点で目標金額以上持っていれば、参加しなくても持って帰ることができるので確率１

・2ラウンド分だけ考えたときに、
　- １ラウンドで目標金額の予分の一未満しか持っていなかったら、勝っても目標に達しないので確率０
　- １ラウンドで目標金額の四分の一以上半分未満をもっていれば、２回勝ったら持って帰ることができるのえｄ、確率P*P
　- １ラウンドで目標金額の半分以上四分の三未満をもっていれば、一回目で半分以上かけて勝てば持って帰ることができるので確率P（一回目で半分以上残して賭けに参加する場合は勝敗がどうであれ結果に影響がないのでどうでもいい）
　- １ラウンドで目標金額の半分以上目標未満をもっていれば、１ラウンドで半分残しで負けて次勝つか、半分以上かけて一回目に勝てば持って帰ることができるので確率P+(1-P)*P
　- その時点で目標金額以上持っていれば、参加しなくても持って帰ることができるので確率１

以上を踏まえると、所持金が 目標金額//(2**M) して、
0 <= x < 目標金額//(2**M) * 1
目標金額//(2**M) * 1 <= x < 目標金額//(2**M) * 2
~~ 中略 ~~
目標金額//(2**M) * (2**M - 1) <= x < 目標金額
目標金額 <= x
で考える
"""


M,X = map(int, input().split())
P = float(input())

# dp, 前回と今回の手が分かればいいので２回分
# それぞれ前回、今回のタイミングで、区分iの金額を持っているときにお金を持ち帰ることができる確率
# 今回の確率は前回の確率にのみ影響される
prev = [0] * (2**M + 1)
curr = [0] * (2**M + 1)

# 前回の手で目標以上に持っている場合は必ずお金を持って帰ることができる
prev[-1] = 1

# Mラウンド実施
# 0ラウンド→1ラウンド→2ラウンド　と逆に辿る
for r in range(M):
    for i in range(2**M + 1):
        """
        

        全ラウンド終了まで残り0ラウンドから逆に辿ること（ゲーム終了→残り1ラウンド→2ラウンド→...のように）によって、残りMラウンド（つまりは最初）の段階で、どの所持金領域にいるときの、お金を持ち帰られる確率が求まる
        2ラウンドあるとして全ラウンド終了時に所持金領域[2]に訪れる方法は、
        ・1ラウンド終了時に所持金領域[2]にいて、0円かけた時
        ・1ラウンド終了時に所持金領域[1]にいて、所持金領域[1]円かけて勝った時
        ・1ラウンド終了時に所持金領域[3]にいて、所持金領域[1]円かけて負けた時
        ・1ラウンド終了時に所持金領域[4]にいて、所持金領域[2]円かけて負けた時
        これを逆に辿る
        """
        for j in range(0, min(i, 2**M-i)+1):
            # 0ラウンド→1ラウンド→2ラウンド　と逆に辿る
            curr[i] = max(curr[i], P * prev[i+j] + (1-P) * prev[i-j])

    prev = curr
    curr = [0] * (2**M + 1)

# 10**6を2**M個に分けたものの中で、Xがどこにあたるかなので、
# X / ( 10**6 / M**2 ) -> X * M**2 / 10**6
idx = int(X * 2**M / (10**6))

print(prev[idx])
